**Автор скрипта**: Кирилл Гулиев

**Проект**: Предсказание революций с помощью методов машинного обучения

In [1]:
# pip install: fuzzywuzzy, fuzzy_pandas (requires PATH rights), Levenshtein 

import pandas as pd
import os
import fuzzywuzzy as fuzz
import fuzzy_pandas as fpd

# Set the current directory to read the data
# NB! Specify your working directory!
os.chdir(r'C:\Users\Кирилл\Desktop\CSR2023\proj_ML_rev\data')

# Dictionary of countries to rename
countries_dict = {
    "Литва": "Lithuania",
    "Латвия": "Latvia",
    "Эстония": "Estonia",
    "Holy See": "Holy See (Vatican City)",
    "Myanmar": "Myanmar (Burma)",
    "Syrian Arab Republic": "Syria",
    "German Democratic Republic": "German DR",
    "Lao People's Democratic Republic": "Laos",
    "Brunei Darussalam": "Brunei",
    "Micronesia" :"Micronesia, Fed States",
    "Viet Nam": "Vietnam",
    "Slovakia": "Slovak Republic",
    "Palestine": "Palestinian Autonomous Areas"
}

# Read the data 
countries_data = pd.read_csv('cnts_2022_3_18.csv', sep = ';')

# Unique country names
country_names = list(set(countries_data['country']))

elections_rawdata = pd.read_csv('Данные по выборам (1).csv', sep = ';')
elections_data = elections_rawdata.rename(columns = \
                                          {'Исследуемая страна (использовать название строго и дословно из CNTS)' : 'country_name'})

# Trim the country names and remove unnecessary string patterns 
elections_data['country_name'] = elections_data['country_name'].str.strip().replace(r'\(.*', '', regex = True) \
    .replace(r'\,.*', '', regex = True) \
    .replace(r'[0-9]..', '', regex = True)

# Final replacement 
elections_data.replace({'country_name' : countries_dict}, inplace = True)

# Match two tables into one
merged_data = fpd.fuzzy_merge(countries_data, elections_data, 
                             left_on = 'country', right_on = 'country_name',
                             method = 'levenshtein', threshold = 0.9)

In [ ]:
# Custom function to merge data by comparing the string distance
# https://stackoverflow.com/a/56315491/12081269
def fuzzy_merge(df_1, df_2, key1, key2, threshold = 90, limit = 2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit = limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1